In [76]:
import os
from scipy.io import loadmat
from PIL import Image
import numpy as np
import torch

In [2]:
import pickle

In [4]:
objs = pickle.load(open("./data/objs_pascal.pkl", "rb"))

In [5]:
import torch.utils.data as data

In [90]:
cls = ('__background__', # always index 0
        'aeroplane', 'bicycle', 'bird', 'boat',
        'bottle', 'bus', 'car', 'cat', 'chair',
        'cow', 'diningtable', 'dog', 'horse',
        'motorbike', 'person', 'pottedplant',
        'sheep', 'sofa', 'train', 'tvmonitor')

class_to_ind = dict(zip(cls, range(len(cls))))

class PasDet(data.Dataset):
    def __init__(self, pickle_file, transform=None, target_transform=None):
        self.objs = pickle_file
        self.ids = list(tuple(self.objs))
    def __getitem__(self, index):
        img_id = self.ids[index]
        img = Image.open('./data/pascal3d+/Images/' + self.objs[img_id][0]['label'] + '/' + img_id + '.jpg')
        bboxes = [self.objs[img_id][0]['bbox']]
        viewpoint = [self.objs[img_id][0]['viewpoint']]
        labels = [class_to_ind[self.objs[img_id][0]['label'].split('_')[0]]]
        for obj in self.objs[img_id][1:]:
            bboxes = np.append(bboxes, obj['bbox'])
            viewpoint = np.append(viewpoint, obj['viewpoint'])
            labels = np.append(labels, class_to_ind[obj['label'].split('_')[0]])
        bboxes = np.reshape(bboxes, [-1, 4]).astype(np.int64)
        return img, {'boxes':torch.LongTensor(bboxes), 'gt_classes': labels, 'viewpoints':viewpoint, 'im_info':list(np.append(img.size, 1))}, img_id
    def __len__(self):
        return len(self.ids)

In [91]:
ds = PasDet(objs)

In [92]:
print(len(ds))

5474
